# Verification de la reprise d'une sauvegarde IJK (format xyz) par TrioCFD (sequentiel et parallele)

In [ ]:
from trustutils import run
# Declaration of Author and (optionally) date 
run.introduction("Guillaume Bois, Téo Boutin","11/02/2025")

# Declaration of the TRUST version
run.TRUST_parameters("1.9.6_beta")


## Disclaimer
DO NOT REPRODUCE THIS REPORT UNLESS YOU REALLY NEED TO !

Use the example from ``` Run_fiche -doc ``` as a reference to make your own validation report.
This report uses low level parts of the trustutils API.

This is necessary because of the unique specifities in this report in order to get a proper extracted test case.

If you need to use these features, first contact a TrioCFD or Trust developper, preferably one who has experience with validation reports and the trustutils API.


## Description

This report only test various restarts in IJK and FTD modules of TrioCFD.

First init.lata is generated from a bash script. It contains an interface with two bubbles.

This init.lata is used in two ijk tests to initialize an interface:
 - reprise_xyz_ijk
 - reprise_xyz_ijk_par8 (runs on 8 procs)

The interface obtained from these two simulations will be converted to med format for use in FTD simulations.

Then, various test cases of FTD module are created, with restarts from the result of the IJK tests, and a reference case:
 - SEQ_reprise_TrioCFD_reference : reference interface
 - SEQ_reprise_TrioCFD_fromSEQIJK : read interface from reprise_xyz_ijk
 - PAR_reprise_TrioCFD_fromSEQIJK : read interface from reprise_xyz_ijk, parallel run
 - SEQ_reprise_TrioCFD_fromPARIJK : read interface from reprise_xyz_ijk_par8
 - PAR_reprise_TrioCFD_fromPARIJK : read interface from reprise_xyz_ijk_par8, parallel run

The results of all these restarts are compared at the end (only velocity fields, as interfaces are specific to FTD and IJK, I believe).

Reminder: FTD interface indic must be written at som and not elem, else wrong interpolations are done during post at borders between multiple procs.


In [ ]:
# This block is the prepare, all tests are actually launched here
# Because we only do restarts, and we want to compare the results of all these reading and rewriting of interfaces.
# We use lower level API of trustutils than usual to achieve what we want
# This is still much better than using a lot of sed in bash scripts...

from trustutils import run
run.reset() 
run.initCaseSuite()
run.executeCommand("./cree_multi_bulles.sh deplacements.txt init.lata")

# Create all the test cases
# they will run in the next cell

# create test case for ijk
reprise_xyz_ijk=run.TRUSTCase(".","reprise_xyz_ijk_template.data").copy("reprise_xyz_ijk.data")
reprise_xyz_ijk.substitute_template({"proc_i":1, "proc_j":1, "proc_k":1})


# create test case for ijk parallel
reprise_xyz_ijk_par8=run.TRUSTCase(".","reprise_xyz_ijk_template.data").copy("reprise_xyz_ijk_par8.data", nbProcs=8)
reprise_xyz_ijk_par8.substitute_template({"proc_i":2, "proc_j":2, "proc_k":2})


# create test cases for FTD restarts from ijk interface
# parallel restarts from med probably wont work
# a last test case will restart FTD in parallel from xyz output of SEQ restart from IJK

# template parameters used to complete the data set
init_interf_med_seq="reprise_xyz_ijk.med"
init_interf_med_par="reprise_xyz_ijk_par8.med"

init_cond_vit_ref="vitesse champ_Fonc_xyz DOM 3 0.2*sin(y*15.707963267949)+31./15.707963267949*cos(z*31.4159265358979)*sin(x*15.707963267949) -0.2*sin(x*15.707963267949) 31./31.4159265358979*sin(z*31.4159265358979)*cos(x*15.707963267949)"
init_cond_vit_seq="vitesse Champ_Fonc_reprise_IJK reprise_xyz_ijk.sauv.xyz pb vitesse"
init_cond_vit_par="vitesse Champ_Fonc_reprise_IJK reprise_xyz_ijk_par8.sauv.xyz pb vitesse"


# reference case for the interface
SEQ_reprise_TrioCFD_reference=run.TRUSTCase(".","reprise_TrioCFD_template.data").copy("SEQ_reprise_TrioCFD_reference.data")


SEQ_reprise_TrioCFD_reference.substitute_template({"init_cond_vit":init_cond_vit_ref, "init_interf_med":init_interf_med_seq})

# restart from the sequential ijk case. will be done in sequential and parallel
reprise_TrioCFD_fromSEQIJK=run.TRUSTCase(".","reprise_TrioCFD_template.data").copy("reprise_TrioCFD_fromSEQIJK.data", nbProcs=8)


reprise_TrioCFD_fromSEQIJK.substitute_template({"init_cond_vit":init_cond_vit_seq, "init_interf_med":init_interf_med_seq})

ok=reprise_TrioCFD_fromSEQIJK.partition(verbose=False)

# restart from the parallel ijk case. will be done in sequential and parallel
reprise_TrioCFD_fromPARIJK=run.TRUSTCase(".","reprise_TrioCFD_template.data").copy("reprise_TrioCFD_fromPARIJK.data", nbProcs=8)

init_interf_med="reprise_xyz_ijk_par8.med"
init_cond="vitesse Champ_Fonc_reprise_IJK reprise_xyz_ijk_par8.sauv.xyz pb vitesse"

reprise_TrioCFD_fromPARIJK.substitute_template({"init_cond_vit":init_cond_vit_par, "init_interf_med":init_interf_med_par})

ok=reprise_TrioCFD_fromPARIJK.partition(verbose=False)


# restart in FTD parallel from a FTD sequential that was restarted from IJK xyz converted to med
#reprise_TrioCFD_fromSEQ_FTD=run.TRUSTCase(".","reprise_TrioCFD_template.data").copy("reprise_TrioCFD_fromSEQ_FTD.data", nbProcs=8)
#reprise_TrioCFD_fromSEQ_FTD.substitute_template({"init_cond_vit":init_cond_vit_par, "init_interf_med":init_interf_med_par})

#reprise_TrioCFD_fromSEQ_FTD.substitute("# resume_last_time #","resume_last_time xyz SEQ_reprise_TrioCFD_fromSEQIJK_pb.xyz")

#ok=reprise_TrioCFD_fromSEQ_FTD.partition(verbose=False)




In [ ]:
# utils for post treatment
# TODO maybe: write an api to create these sed commands. This is unreadable...
def post_treatment_ijk(jdd):
    # IJK velocity field to compare
    cmd="sed '/INTERFACES/d' {jdd}.sauv.lata > {jdd}_velocity.lata".format(jdd=jdd)
    run.executeCommand(cmd)
    # IJK interf to compare with FTD interf
    cmd="sed -e '/IJK_EXT/d' -e '/FACETTE_PE/d' -e '/COMPO_CONNEXE/d'  -e '/INDEX_SOMMET_REEL/d' {jdd}.lata > {jdd}_interf.lata".format(jdd=jdd)
    run.executeCommand(cmd)
    cmd="source $TRUST_ROOT/env_for_python.sh && python $TRUST_ROOT/exec/lata_tools/lib/Lata_to_med.py {jdd}.sauv.lata {jdd}.med INTERFACES 1".format(jdd=jdd)
    run.executeCommand(cmd)

def post_treatment_ftd(jdd):
    cmd="sed -e '/INTERFACES/d' -e '/INDICATRICE_INTERF/d' {jdd}_POST1.lata > {jdd}_velocity.lata".format(jdd=jdd)
    run.executeCommand(cmd)
    # remove geoms from FTD ref to be able to compare velocity to IJK
    cmd="sed -e '/DOM_boundaries_bas/d' -e '/DOM_boundaries_haut/d' -e '/DOM_boundaries_perioY/d' -e '/DOM_boundaries_perioX/d' {jdd}_velocity.lata > {jdd}_velocity_no_geom.lata".format(jdd=jdd)
    run.executeCommand(cmd)
    # to compare FTD interf with IJK interf
    cmd="sed -e '/COMPO_CONNEXE/d' -e '/PE/d' -e '/PE_LOCAL/d' -e '/INDICATRICE_INTERF_SOM_DOM/d' -e '/DOM_boundaries_bas/d' -e '/DOM_boundaries_haut/d' -e '/DOM_boundaries_perioY/d' -e '/DOM_boundaries_perioX/d' {jdd}_POST1.lata > {jdd}_interf.lata".format(jdd=jdd)
    run.executeCommand(cmd)

# run the test cases

# run ijk sequential restart
run.executeCommand("trust reprise_xyz_ijk.data")

post_treatment_ijk("reprise_xyz_ijk")

# run ijk parallel restart
run.executeCommand("trust reprise_xyz_ijk_par8.data 8")

post_treatment_ijk("reprise_xyz_ijk_par8")


# run ftd cases

run.executeCommand("trust SEQ_reprise_TrioCFD_reference")
post_treatment_ftd("SEQ_reprise_TrioCFD_reference")

run.executeCommand("trust SEQ_reprise_TrioCFD_fromSEQIJK")
post_treatment_ftd("SEQ_reprise_TrioCFD_fromSEQIJK")

run.executeCommand("trust SEQ_reprise_TrioCFD_fromPARIJK")
post_treatment_ftd("SEQ_reprise_TrioCFD_fromPARIJK")

run.executeCommand("trust PAR_reprise_TrioCFD_fromSEQIJK 8")
post_treatment_ftd("PAR_reprise_TrioCFD_fromSEQIJK")

run.executeCommand("trust PAR_reprise_TrioCFD_fromPARIJK 8")
post_treatment_ftd("PAR_reprise_TrioCFD_fromPARIJK")



In [ ]:
# verifications using compare_lata for the test case
# compare velocity fields between all cases
from trustutils import run
def compare_lata(lata1, lata2, options=""):
    cmd="compare_lata %s %s %s" % (lata1, lata2, options)
    run.executeCommand(cmd, verbose=True)
    
# compare IJK seq to par
compare_lata("reprise_xyz_ijk_velocity.lata", "reprise_xyz_ijk_par8_velocity.lata")

# compare FTD seq to par in restart from IJS seq
compare_lata("SEQ_reprise_TrioCFD_fromSEQIJK_velocity.lata", "PAR_reprise_TrioCFD_fromSEQIJK_velocity.lata")

# compare FTD seq to par in restart from IJS par
compare_lata("SEQ_reprise_TrioCFD_fromPARIJK_velocity.lata", "PAR_reprise_TrioCFD_fromPARIJK_velocity.lata")

# compare FTD seq from IJK seq to FTD seq from IJK par
compare_lata("SEQ_reprise_TrioCFD_fromSEQIJK_velocity.lata", "SEQ_reprise_TrioCFD_fromPARIJK_velocity.lata")

# compare FTD seq to reference
compare_lata("SEQ_reprise_TrioCFD_fromSEQIJK_velocity.lata", "SEQ_reprise_TrioCFD_reference_velocity.lata")

# compare IJK seq to FTD ref
compare_lata("reprise_xyz_ijk_velocity.lata", "SEQ_reprise_TrioCFD_reference_velocity_no_geom.lata")

# compare interfaces from IJK seq to FTD seq
compare_lata("reprise_xyz_ijk_interf.lata", "SEQ_reprise_TrioCFD_fromSEQIJK_interf.lata", options="")

In [ ]:
# dummy test case added through the normal procedure to ensure a test case can be extracted
# must be parallel so the test is allocated 8 procs
run.addCase(".", "reprise_xyz_ijk_par8.data", nbProcs=8)

# probably needed so the extraction of test stops at the correct time
run.runCases()

## Plots of interfaces

In [ ]:
from trustutils import visit
# plot IJK
visit.showMesh("reprise_xyz_ijk.lata","INTERFACES")

visit.showMesh("reprise_xyz_ijk_par8.lata","INTERFACES")

# plot FTD
visit.showMesh("SEQ_reprise_TrioCFD_reference_POST1.lata","INTERFACES")
visit.showField("SEQ_reprise_TrioCFD_reference_POST1.lata","Contour","INDICATRICE_INTERF_SOM_DOM",plotmesh=False)

visit.showMesh("SEQ_reprise_TrioCFD_fromSEQIJK_POST1.lata","INTERFACES")
visit.showField("SEQ_reprise_TrioCFD_fromSEQIJK_POST1.lata","Contour","INDICATRICE_INTERF_SOM_DOM",plotmesh=False)

visit.showMesh("SEQ_reprise_TrioCFD_fromPARIJK_POST1.lata","INTERFACES")
visit.showField("SEQ_reprise_TrioCFD_fromPARIJK_POST1.lata","Contour","INDICATRICE_INTERF_SOM_DOM",plotmesh=False)

visit.showMesh("PAR_reprise_TrioCFD_fromSEQIJK_POST1.lata","INTERFACES")
visit.showField("PAR_reprise_TrioCFD_fromSEQIJK_POST1.lata","Contour","INDICATRICE_INTERF_SOM_DOM",plotmesh=False)

visit.showMesh("PAR_reprise_TrioCFD_fromPARIJK_POST1.lata","INTERFACES")
visit.showField("PAR_reprise_TrioCFD_fromPARIJK_POST1.lata","Contour","INDICATRICE_INTERF_SOM_DOM",plotmesh=False)